# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
vacation= pd.read_csv("../output_data/cities.csv")
vacation


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [49]:
# Store latitude and longitude in locations
locations = vacation[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = vacation["Humidity"]

#make heatmap
fig = gmaps.figure(center=(36.8529, -75.978), zoom_level=2)

#make heatmap
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity,  max_intensity=100, 
dissipating=False, point_radius = 2))

# Display Figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
# A max temperature lower than 80 degrees but higher than 70.

max_temp = vacation.loc[(vacation["Max Temp"] > 70) & (vacation["Max Temp"] < 80)]
max_temp


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
18,18,bambous virieux,75,MU,1558378758,74,-20.34,57.76,78.80,11.41
25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93
...,...,...,...,...,...,...,...,...,...,...
516,516,malindi,75,KE,1558378808,88,-3.22,40.12,77.00,9.17
521,521,high rock,75,BS,1558378893,83,26.63,-78.28,75.20,12.75
530,530,hirara,37,JP,1558378895,67,24.80,125.28,75.48,26.15
534,534,marzuq,84,YE,1558378896,36,14.40,46.47,79.44,6.02


In [36]:
 #Wind speed less than 10 mph.
wind_speed = max_temp.loc[(max_temp["Wind Speed"] < 10) ]
wind_speed.head(5)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93
44,44,puerto ayora,26,EC,1558378762,86,-0.74,-90.35,79.00,5.99
59,59,bontang,54,ID,1558378765,99,0.12,117.47,74.22,2.62


In [37]:
#Zero cloudiness
zero_cloudiness = wind_speed.loc[(wind_speed["Cloudiness"] ==0) ]
zero_cloudiness.head(5)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
# Create a hotel_df
hotel_df = zero_cloudiness.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
7,opuwo,NaN,-18.06,13.84,
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
297,mogok,MM,22.92,96.51,
444,birjand,IR,32.86,59.22,


In [43]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [45]:
# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    if row["Hotel Name"]=="":

        # change location each iteration while leaving original params in place
        params["location"] = f"{lat},{lng}"

        # make request and print url
        name = requests.get(base_url, params=params).json()
        

        hotel_df.loc[index, "Hotel Name"] = name["results"][0]["name"]

hotel_df

,City,Country,Lat,Lng,Hotel Name
7,opuwo,NaN,-18.06,13.84,Kaoko Mopane Lodge & Campsite | Opuwo
70,ponta do sol,BR,-20.63,-46.00,Escarpas Resort
88,cidreira,BR,-30.17,-50.22,Hotel Castelo
138,morondava,MG,-20.30,44.28,Palissandre Cote Ouest resort & SPA
139,vaini,IN,15.34,74.49,Dandeli Dreams Jungle Resort
176,nador,MA,35.17,-2.93,Hôtel Mercure Rif Nador
297,mogok,MM,22.92,96.51,MOGOK HILL HOTEL
444,birjand,IR,32.86,59.22,هتل پرويز


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [47]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig



Figure(layout=FigureLayout(height='420px'))